In [1]:
import os
import sys 
os.chdir('..')
sys.path.insert(0, os.getcwd())
import numpy as np
import tensorflow as tf
import glob

from Data_loader import load_subjects_from_json, get_all_npy_paths_by_group, base_folders
from AE_pipeline import (
    convert_npy_to_tfrecord,
    create_tfrecord_dataset,
    write_sharded_tfrecord,
    make_monolithic_ds,
    build_lstm_autoencoder,
    train_autoencoder,
    evaluate_and_detect,
    extract_and_save_latents,
    n_timesteps,
    NUM_BIOMECHANICAL_VARIABLES,
    _parse_cycle,
    BATCH_SIZE
)

2025-06-10 21:57:27.527515: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-10 21:57:27.614861: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-10 21:57:27.683372: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-10 21:57:27.754941: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-10 21:57:27.776485: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-10 21:57:27.893864: I tensorflow/core/platform/cpu_feature_gu

Mixed precision enabled


In [2]:
#Load subjects lists 
train_subjects = {
    "G01": load_subjects_from_json("G01_train_subjects.json"),
    "G03": load_subjects_from_json("G03_train_subjects.json")
}
val_subjects = {
    "G01": load_subjects_from_json("G01_validation_subjects.json"),
    "G03": load_subjects_from_json("G03_validation_subjects.json")
}
test_subjects = {
    "G01": load_subjects_from_json("G01_test_subjects.json"),
    "G03": load_subjects_from_json("G03_test_subjects.json")
}


In [3]:
# Generate routes .npy
train_npy = get_all_npy_paths_by_group(train_subjects, base_folders)
val_npy   = get_all_npy_paths_by_group(val_subjects,   base_folders)
test_npy  = get_all_npy_paths_by_group(test_subjects,  base_folders)

print(f"Train .npy: {len(train_npy)} files")
print(f" Val  .npy: {len(val_npy)} files")
print(f" Test .npy: {len(test_npy)} files")


Train .npy: 932 files
 Val  .npy: 196 files
 Test .npy: 169 files


In [4]:
#Verifying NaN/Inf and removing is found 
bad = []
for p in train_npy:
    arr = np.load(p).astype(np.float32)
    if not np.isfinite(arr).all():
        bad.append(p)
print("Encontré NaN/Inf en:", bad)


Encontré NaN/Inf en: ['/mnt/storage/dmartinez/young adults (19–35 years old)/S030/preprocessed/S030_D01_B03_T03_preprocessed.npy', '/mnt/storage/dmartinez/young adults (19–35 years old)/S031/preprocessed/S031_D02_B02_T03_preprocessed.npy', '/mnt/storage/dmartinez/young adults (19–35 years old)/S016/preprocessed/S016_D01_B02_T01_preprocessed.npy', '/mnt/storage/dmartinez/old adults (56+ years old)/S040/preprocessed/S040_D01_B01_T03_preprocessed.npy', '/mnt/storage/dmartinez/old adults (56+ years old)/S158/preprocessed/S158_D02_B01_T02_preprocessed.npy']


In [12]:
#verificando si fue despues del prepro que salen esos NaN 

bad_npy_paths = [
    '/mnt/storage/dmartinez/young adults (19–35 years old)/S030/preprocessed/S030_D01_B03_T03_preprocessed.npy',
    '/mnt/storage/dmartinez/young adults (19–35 years old)/S031/preprocessed/S031_D02_B02_T03_preprocessed.npy',
    '/mnt/storage/dmartinez/young adults (19–35 years old)/S016/preprocessed/S016_D01_B02_T01_preprocessed.npy',
    '/mnt/storage/dmartinez/old adults (56+ years old)/S040/preprocessed/S040_D01_B01_T03_preprocessed.npy',
    '/mnt/storage/dmartinez/old adults (56+ years old)/S158/preprocessed/S158_D02_B01_T02_preprocessed.npy'
]

for p in bad_npy_paths:
    print(f"\nAnalizando: {p}")
    arr_orig = np.load(p)
    print(f"  Shape original: {arr_orig.shape}")
    print(f"  Tipo de dato original: {arr_orig.dtype}")
    print(f"  ¿Contiene NaN/Inf directamente?: {~np.isfinite(arr_orig).all()}")
    if not np.isfinite(arr_orig).all():
        # Si hay NaN/Inf, busca dónde están
        nan_indices = np.argwhere(np.isnan(arr_orig))
        inf_indices = np.argwhere(np.isinf(arr_orig))
        print(f"    Índices de NaN: {nan_indices}")
        print(f"    Índices de Inf: {inf_indices}")
        # Puedes incluso imprimir algunos valores alrededor de esos índices si son pocos
        # print(f"    Valores en índices de NaN: {arr_orig[nan_indices[0][0], nan_indices[0][1], nan_indices[0][2]] if len(nan_indices) > 0 else 'N/A'}")
        # print(f"    Valores en índices de Inf: {arr_orig[inf_indices[0][0], inf_indices[0][1], inf_indices[0][2]] if len(inf_indices) > 0 else 'N/A'}")

    # Verifica si hay columnas con desviación estándar o rango cero
    # Asegúrate de que arr_orig esté en la forma correcta para estas comprobaciones
    # Asumimos que la forma es (n_ciclos, n_timesteps, n_variables)
    if arr_orig.ndim == 3:
        # Calcular std/min/max por variable sobre todos los ciclos y timesteps para detectar columnas constantes
        # Flatten los timesteps para agrupar todas las observaciones de una variable
        arr_reshaped = arr_orig.reshape(-1, arr_orig.shape[2])
        std_devs = np.std(arr_reshaped, axis=0)
        ranges = np.max(arr_reshaped, axis=0) - np.min(arr_reshaped, axis=0)

        zero_std_cols = np.where(std_devs < 1e-9)[0] # Usar un pequeño epsilon para float comparisons
        zero_range_cols = np.where(ranges < 1e-9)[0]

        if len(zero_std_cols) > 0:
            print(f"  Columnas con desviación estándar cercana a cero: {zero_std_cols}")
        if len(zero_range_cols) > 0:
            print(f"  Columnas con rango cercano a cero: {zero_range_cols}")

        # Verifica si hay valores extremadamente grandes o pequeños
        print(f"  Valor mínimo: {np.min(arr_orig[np.isfinite(arr_orig)]) if np.isfinite(arr_orig).any() else 'No finite values'}")
        print(f"  Valor máximo: {np.max(arr_orig[np.isfinite(arr_orig)]) if np.isfinite(arr_orig).any() else 'No finite values'}")


Analizando: /mnt/storage/dmartinez/young adults (19–35 years old)/S030/preprocessed/S030_D01_B03_T03_preprocessed.npy
  Shape original: (200, 100, 326)
  Tipo de dato original: float64
  ¿Contiene NaN/Inf directamente?: True
    Índices de NaN: [[ 72  27 117]
 [ 72  27 118]
 [ 72  27 119]
 [ 72  27 120]
 [ 72  27 121]
 [ 72  27 122]
 [ 72  28 117]
 [ 72  28 118]
 [ 72  28 119]
 [ 72  28 120]
 [ 72  28 121]
 [ 72  28 122]
 [ 79  44 117]
 [ 79  44 118]
 [ 79  44 119]
 [ 79  44 120]
 [ 79  44 121]
 [ 79  44 122]
 [ 79  45 117]
 [ 79  45 118]
 [ 79  45 119]
 [ 79  45 120]
 [ 79  45 121]
 [ 79  45 122]
 [ 79  46 117]
 [ 79  46 118]
 [ 79  46 119]
 [ 79  46 120]
 [ 79  46 121]
 [ 79  46 122]
 [ 79  79 117]
 [ 79  79 118]
 [ 79  79 119]
 [ 79  79 120]
 [ 79  79 121]
 [ 79  79 122]
 [ 79  80 117]
 [ 79  80 118]
 [ 79  80 119]
 [ 79  80 120]
 [ 79  80 121]
 [ 79  80 122]
 [ 80   7 117]
 [ 80   7 118]
 [ 80   7 119]
 [ 80   7 120]
 [ 80   7 121]
 [ 80   7 122]
 [ 80   8 117]
 [ 80   8 118]
 [ 8

In [11]:
#To remove the cycles with NaN/Inf from the identified trials 
def filter_cycles_in_file(path):
    arr = np.load(path).astype(np.float32)
    mask = np.isfinite(arr).all(axis=(1,2))
    if not mask.any():
        return None  # todo el archivo está mal
    # guarda un .npy limpio temporal si quieres:
    clean = arr[mask]
    tmp_path = path.replace("_preprocessed.npy", "_filtered.npy")
    np.save(tmp_path, clean)
    return tmp_path

# Ejemplo aplicándolo a train_npy:
filtered_paths = []
for p in train_npy:
    newp = filter_cycles_in_file(p)
    if newp:
        filtered_paths.append(newp)
train_npy = filtered_paths
print("→ Archivos reescritos sin ciclos corruptos:", len(train_npy))


KeyboardInterrupt: 

In [4]:
#Generar SHARDS para TRAIN 
shards_dir = "train_shards"
if not os.path.isdir(shards_dir):
    write_sharded_tfrecord(
        npy_paths=train_npy,
        output_dir=shards_dir,
        shard_size=100_000
    )
    print(f"→ Shards generados en: {shards_dir}/")
else:
    print(f"→ Shards ya existen en: {shards_dir}/")

→ Shards ya existen en: train_shards/


In [5]:
#Convertir VAL y TEST a TFRecord monolítico 
for split, npy_list in [("val", val_npy), ("test", test_npy)]:
    tfp = f"{split}_cycles.tfrecord.gz"
    if not os.path.exists(tfp):
        convert_npy_to_tfrecord(npy_list, tfp)
        print(f"Converted → {tfp}")
    else:
        print(f"Skipping (already exists) → {tfp}")

Skipping (already exists) → val_cycles.tfrecord.gz
Skipping (already exists) → test_cycles.tfrecord.gz


In [6]:
#Create tf.data.Dataset
# 3a) Lista de archivos shard
shard_files = sorted(glob.glob(os.path.join(shards_dir, "*.tfrecord.gz")))

# 3b) Pipeline shard-aware
train_ds = (
   tf.data.Dataset
      .list_files(shard_files, shuffle=True)
      .interleave(
         lambda f: tf.data.TFRecordDataset(f, compression_type="GZIP"),
         cycle_length=4,
         num_parallel_calls=tf.data.AUTOTUNE
      )
      .map(_parse_cycle, num_parallel_calls=tf.data.AUTOTUNE)
      .shuffle(5_000, seed=42)
      .batch(BATCH_SIZE, drop_remainder=True)
      .prefetch(tf.data.AUTOTUNE)
)

# VAL y TEST: dataset monolítico


val_ds  = make_monolithic_ds("val_cycles.tfrecord.gz")
test_ds = make_monolithic_ds("test_cycles.tfrecord.gz")

print(f"→ train_ds: {train_ds}")
print(f"→ val_ds:   {val_ds}")
print(f"→ test_ds:  {test_ds}")


→ train_ds: <_PrefetchDataset element_spec=(TensorSpec(shape=(32, 100, 321), dtype=tf.float32, name=None), TensorSpec(shape=(32, 100, 321), dtype=tf.float32, name=None))>
→ val_ds:   <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 100, 321), dtype=tf.float32, name=None), TensorSpec(shape=(None, 100, 321), dtype=tf.float32, name=None))>
→ test_ds:  <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 100, 321), dtype=tf.float32, name=None), TensorSpec(shape=(None, 100, 321), dtype=tf.float32, name=None))>


I0000 00:00:1749549034.051885  118688 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-06-10 06:50:34.053001: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [10]:
# Toma un batch de train_ds
for x_batch, y_batch in train_ds.take(1):
    import tensorflow as tf
    print("Input  ◂ min:", tf.reduce_min(x_batch).numpy(),
          "max:", tf.reduce_max(x_batch).numpy(),
          "mean:", tf.reduce_mean(x_batch).numpy(),
          "std:", tf.math.reduce_std(x_batch).numpy())
    print("Target ◂", 
          tf.reduce_min(y_batch).numpy(), tf.reduce_max(y_batch).numpy())
    # Comprueba si hay NaN/Inf
    print("Any NaN in x?", tf.reduce_any(tf.math.is_nan(x_batch)).numpy())
    print("Any Inf in x?", tf.reduce_any(tf.math.is_inf(x_batch)).numpy())
    break


Input  ◂ min: -16.053112 max: 12.33923 mean: 0.04029422 std: 0.9595462
Target ◂ -16.053112 12.33923
Any NaN in x? False
Any Inf in x? False


In [7]:
#original 
#Build and train the Autoencoder
# Hiperparameters 
latent_dim = 64
epochs     = 100
lr = 1e-5
model = build_lstm_autoencoder(N_TIMESTEPS, NUM_BIOMECHANICAL_VARIABLES, latent_dim, lr)
history = train_autoencoder(model, train_ds, val_ds, epochs=epochs)


Epoch 1/100
   6086/Unknown 611s 99ms/step - loss: nan

2025-06-10 07:00:47.014153: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/home/dmartinez/miniconda3/envs/inv_Di/lib/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


6086/6086 ━━━━━━━━━━━━━━━━━━━━ 655s 106ms/step - loss: nan - val_loss: nan - learning_rate: 1.0000e-05
Epoch 2/100


2025-06-10 07:01:31.677230: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


6086/6086 ━━━━━━━━━━━━━━━━━━━━ 651s 107ms/step - loss: nan - val_loss: nan - learning_rate: 1.0000e-05
Epoch 3/100


...

6086/6086 ━━━━━━━━━━━━━━━━━━━━ 617s 101ms/step - loss: nan - val_loss: nan - learning_rate: 1.0000e-05
Epoch 4/100
6086/6086 ━━━━━━━━━━━━━━━━━━━━ 623s 102ms/step - loss: nan - val_loss: nan - learning_rate: 1.0000e-05
Epoch 5/100


2025-06-10 07:33:02.190448: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


6086/6086 ━━━━━━━━━━━━━━━━━━━━ 622s 102ms/step - loss: nan - val_loss: nan - learning_rate: 1.0000e-05
Epoch 6/100
6086/6086 ━━━━━━━━━━━━━━━━━━━━ 618s 101ms/step - loss: nan - val_loss: nan - learning_rate: 5.0000e-06
Epoch 7/100
6086/6086 ━━━━━━━━━━━━━━━━━━━━ 618s 101ms/step - loss: nan - val_loss: nan - learning_rate: 5.0000e-06
Epoch 8/100
6086/6086 ━━━━━━━━━━━━━━━━━━━━ 619s 101ms/step - loss: nan - val_loss: nan - learning_rate: 5.0000e-06
Epoch 9/100


2025-06-10 08:14:19.541030: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


6086/6086 ━━━━━━━━━━━━━━━━━━━━ 618s 101ms/step - loss: nan - val_loss: nan - learning_rate: 5.0000e-06
Epoch 10/100
6086/6086 ━━━━━━━━━━━━━━━━━━━━ 619s 101ms/step - loss: nan - val_loss: nan - learning_rate: 5.0000e-06
Epoch 11/100
6086/6086 ━━━━━━━━━━━━━━━━━━━━ 622s 102ms/step - loss: nan - val_loss: nan - learning_rate: 2.5000e-06


In [8]:
#Evaluation
all_losses, threshold = evaluate_and_detect(model, test_ds)


Test reconstruction MSE: nan


TypeError: in user code:

    File "/home/dmartinez/Gait-Stability/AE_pipeline.py", line 180, in None  *
        lambda x, _: tf.reduce_mean(tf.math.squared_difference(x, model(x)), axis=[1,2])

    TypeError: Input 'y' of 'SquaredDifference' Op has type float16 that does not match type float32 of argument 'x'.


In [ ]:
#Latent characteristics 
latents = extract_and_save_latents(model, test_ds, output_path="latent_features_test.npy")
